In [1]:
"""
A class is made up of methods and state. This allows code and data to be
combined as one logical entity. This module defines a basic car class,
creates a car instance and uses it for demonstration purposes.
"""
from inspect import isfunction, ismethod, signature


class Car:
    """Basic definition of a car.

    We begin with a simple mental model of what a car is. That way, we
    can start exploring the core concepts that are associated with a
    class definition.
    """

    def __init__(self, make, model, year, miles):
        """Constructor logic."""
        self.make = make
        self.model = model
        self.year = year
        self.miles = miles

    def __repr__(self):
        """Formal representation for developers."""
        return f"<Car make={self.make} model={self.model} year={self.year}>"

    def __str__(self):
        """Informal representation for users."""
        return f"{self.make} {self.model} ({self.year})"

    def drive(self, rate_in_mph):
        """Drive car at a certain rate in MPH."""
        return f"{self} is driving at {rate_in_mph} MPH"


def main():
    # Create a car with the provided class constructor
    car = Car("Bumble", "Bee", 2000, 200000.0)

    # Formal representation is good for debugging issues
    assert repr(car) == "<Car make=Bumble model=Bee year=2000>"

    # Informal representation is good for user output
    assert str(car) == "Bumble Bee (2000)"

    # Call a method on the class constructor
    assert car.drive(75) == "Bumble Bee (2000) is driving at 75 MPH"

    # As a reminder: everything in Python is an object! And that applies
    # to classes in the most interesting way - because they're not only
    # subclasses of object - they are also instances of object. This
    # means that we can modify the `Car` class at runtime, just like any
    # other piece of data we define in Python
    assert issubclass(Car, object) and isinstance(Car, object)

    # To emphasize the idea that everything is an object, let's look at
    # the `drive` method in more detail
    driving = getattr(car, "drive")

    # The variable method is the same as the instance method
    assert driving == car.drive

    # The variable method is bound to the instance
    assert driving.__self__ == car

    # That is why `driving` is considered a method and not a function
    assert ismethod(driving) and not isfunction(driving)

    # And there is only one parameter for `driving` because `__self__`
    # binding is implicit
    driving_params = signature(driving).parameters
    assert len(driving_params) == 1
    assert "rate_in_mph" in driving_params


if __name__ == "__main__":
    main()

In [2]:
"""
Abstract class is an extension of a basic class. Like a basic class, an
abstract class has methods and state. Unlike a basic class, it inherits
the `ABC` class and has at least one `abstractmethod`. That means we
cannot create an instance directly from its constructor. In this module,
we will create an abstract class and two concrete classes.

For more about abstract classes, click the link below:

https://www.python.org/dev/peps/pep-3119/
"""
from abc import ABC, abstractmethod


class Employee(ABC):
    """Abstract definition of an employee.

    Any employee can work and relax. The way that one type of employee
    can work and relax is different from another type of employee.
    """

    def __init__(self, name, title):
        self.name = name
        self.title = title

    def __str__(self):
        return self.name

    @abstractmethod
    def do_work(self):
        """Do something for work."""
        raise NotImplementedError

    @abstractmethod
    def do_relax(self):
        """Do something to relax."""
        raise NotImplementedError


class Engineer(Employee):
    """Concrete definition of an engineer.

    The Engineer class is concrete because it implements every
    `abstractmethod` that was not implemented above.

    Notice that we leverage the parent's constructor when creating
    this object. We also define `do_refactor` for an engineer, which
    is something that a manager prefers not to do.
    """

    def __init__(self, name, title, skill):
        super().__init__(name, title)
        self.skill = skill

    def do_work(self):
        return f"{self} is coding in {self.skill}"

    def do_relax(self):
        return f"{self} is watching YouTube"

    def do_refactor(self):
        """Do the hard work of refactoring code, unlike managers."""
        return f"{self} is refactoring code"


class Manager(Employee):
    """Concrete definition of a manager.

    The Manager class is concrete for the same reasons as the Engineer
    class is concrete. Notice that a manager has direct reports and
    has the responsibility of hiring people on the team, unlike an
    engineer.
    """

    def __init__(self, name, title, direct_reports):
        super().__init__(name, title)
        self.direct_reports = direct_reports

    def do_work(self):
        return f"{self} is meeting up with {len(self.direct_reports)} reports"

    def do_relax(self):
        return f"{self} is taking a trip to the Bahamas"

    def do_hire(self):
        """Do the hard work of hiring employees, unlike engineers."""
        return f"{self} is hiring employees"


def main():
    # Declare two engineers
    engineer_john = Engineer("John Doe", "Software Engineer", "Android")
    engineer_jane = Engineer("Jane Doe", "Software Engineer", "iOS")
    engineers = [engineer_john, engineer_jane]

    # These engineers are employees but not managers
    assert all(isinstance(engineer, Employee) for engineer in engineers)
    assert all(not isinstance(engineer, Manager) for engineer in engineers)

    # Engineers can work, relax and refactor
    assert engineer_john.do_work() == "John Doe is coding in Android"
    assert engineer_john.do_relax() == "John Doe is watching YouTube"
    assert engineer_john.do_refactor() == "John Doe is refactoring code"

    # Declare manager with engineers as direct reports
    manager_max = Manager("Max Doe", "Engineering Manager", engineers)

    # Managers are employees but not engineers
    assert isinstance(manager_max, Employee)
    assert not isinstance(manager_max, Engineer)

    # Managers can work, relax and hire
    assert manager_max.do_work() == "Max Doe is meeting up with 2 reports"
    assert manager_max.do_relax() == "Max Doe is taking a trip to the Bahamas"
    assert manager_max.do_hire() == "Max Doe is hiring employees"


if __name__ == "__main__":
    main()

In [3]:
"""
Exception classes are used to indicate that something has gone wrong with
the program at runtime. Functions use the `raise` keyword, if an error is
anticipated, and specify the exception class they intend to throw. This
module defines a handful of custom exception classes and shows how they
can be used in the context of a function.
"""


class CustomError(Exception):
    """Custom class of errors.

    This is a custom exception for any issues that arise in this module.
    One of the reasons why developers design a class like this is for
    consumption by downstream services and command-line tools.

    If we designed a standalone application with no downstream consumers, then
    it makes little sense to define a custom hierarchy of exceptions. In that
    case, we should use the existing hierarchy of builtin exception
    classes which are listed in the Python docs:

    https://docs.python.org/3/library/exceptions.html
    """


class DivisionError(CustomError):
    """Any division error that results from invalid input.

    This exception can be subclassed with the following exceptions if they
    happen enough across the codebase:

    - ZeroDivisorError
    - NegativeDividendError
    - NegativeDivisorError

    That being said, there's a point of diminishing returns when we design
    too many exceptions. It is better to design few exceptions that many
    developers handle than design many exceptions that few developers handle.
    """


def divide_positive_numbers(dividend, divisor):
    """Divide a positive number by another positive number.

    Writing a program in this style is considered defensive programming.
    For more on this programming style, check the Wikipedia link below:

    https://en.wikipedia.org/wiki/Defensive_programming
    """
    if dividend <= 0:
        raise DivisionError(f"Non-positive dividend: {dividend}")
    elif divisor <= 0:
        raise DivisionError(f"Non-positive divisor: {divisor}")
    return dividend // divisor


def main():
    # Exception classes are no different from concrete classes in that
    # they all have inheritance baked in
    assert issubclass(DivisionError, CustomError)

    # Try a couple of inputs that are known to throw an error based on
    # the exceptions thrown in `divide_positive_numbers`
    for dividend, divisor in [(0, 1), (1, 0), (-1, 1), (1, -1)]:
        division_failed = False
        try:
            divide_positive_numbers(dividend, divisor)
        except DivisionError as e:
            division_failed = True
            assert str(e).startswith("Non-positive")
        assert division_failed is True

    # Now let's do it correctly to skip all the exceptions
    result = divide_positive_numbers(1, 1)
    assert result == 1


if __name__ == "__main__":
    main()

In [4]:
"""
Iterator classes implement the `__iter__` and `__next__` magic methods.
This module defines an employee iterator class that iterates through each
employee in a hierarchy one-by-one. This module also shows how a similar
approach can be achieved with a generator function.
"""

# Module-level constants
_ITERATION_MESSAGE = "Cyclic loop detected"


class Employee:
    """Generic employee class.

    For this module, we're going to remove the inheritance hierarchy
    in `abstract_class` and make all employees have a `direct_reports`
    attribute.

    Notice that if we continue adding employees in the `direct_reports`
    attribute, those same employees have a `direct_reports` attribute
    as well.

    The tree-like structure of this class resembles the Composite design
    pattern, and it can be found on Wikipedia:

    https://en.wikipedia.org/wiki/Composite_pattern

    Design patterns are battle-tested ways of structuring code to handle
    common problems encountered while writing software in a team setting.
    Here's a Wikipedia link for more design patterns:

    https://en.wikipedia.org/wiki/Design_Patterns
    """

    def __init__(self, name, title, direct_reports):
        self.name = name
        self.title = title
        self.direct_reports = direct_reports


class IterationError(RuntimeError):
    """Any error that comes while iterating through objects.

    Notice that this class inherits from `RuntimeError`. That way dependent
    functions can handle this exception using either the package hierarchy
    or the native hierarchy.
    """


class EmployeeIterator:
    """Employee iterator.

    An iterator class is composed of three methods:

    - A constructor which defines data structures
    - An iterator returns the instance itself
    - A retriever which gets the next element

    We do this by providing what are called magic methods. Other people
    call them d-under methods because they have double-underscores.

    An iterator class resembles the Iterator design pattern, and it
    can be found on Wikipedia:

    https://en.wikipedia.org/wiki/Iterator_pattern
    """

    def __init__(self, employee):
        """Constructor logic."""
        self.employees_to_visit = [employee]
        self.employees_visited = set()

    def __iter__(self):
        """Iterator is self by convention."""
        return self

    def __next__(self):
        """Return the next employee available.

        The logic may seem complex, but it's actually a common algorithm
        used in traversing a relationship graph. It is called depth-first
        search and it can be found on Wikipedia:

        https://en.wikipedia.org/wiki/Depth-first_search
        """
        if not self.employees_to_visit:
            raise StopIteration
        employee = self.employees_to_visit.pop()
        if employee.name in self.employees_visited:
            raise IterationError(_ITERATION_MESSAGE)
        self.employees_visited.add(employee.name)
        for report in employee.direct_reports:
            self.employees_to_visit.append(report)
        return employee


def employee_generator(top_employee):
    """Employee generator.

    It is essentially the same logic as above except constructed as a
    generator function. Notice that the generator code is in a single
    place, whereas the iterator code is in multiple places. Also notice
    that we are using the `yield` keyword in the generator code.

    It is a matter of preference and context that we choose one approach
    over the other. If we want something simple, go with the generator.
    Otherwise, go with the iterator to fulfill more demanding requirements.
    In this case, examples of such requirements are tasks like encrypting
    the employee's username, running statistics on iterated employees or
    excluding the reports under a particular set of managers.

    For more on the subject of using a function versus a class, check
    out this post from Microsoft Developer Blogs:

    https://devblogs.microsoft.com/python/idiomatic-python-functions-versus-classes/
    """
    to_visit = [top_employee]
    visited = set()
    while len(to_visit) > 0:
        employee = to_visit.pop()
        if employee.name in visited:
            raise IterationError(_ITERATION_MESSAGE)
        visited.add(employee.name)
        for report in employee.direct_reports:
            to_visit.append(report)
        yield employee


def main():
    # Manager with two direct reports
    manager = Employee("Max Doe", "Engineering Manager", [
        Employee("John Doe", "Software Engineer", []),
        Employee("Jane Doe", "Software Engineer", [])
    ])

    # We should provide the same three employees in the same order regardless
    # of whether we use the iterator class or the generator function
    employees = [emp for emp in EmployeeIterator(manager)]
    assert employees == [emp for emp in employee_generator(manager)]
    assert len(employees) == 3

    # Make sure that the employees are who we expect them to be
    assert all(isinstance(emp, Employee) for emp in employees)

    # This is not a good day for this company
    hacker = Employee("Unknown", "Hacker", [])
    hacker.direct_reports.append(hacker)

    for iter_obj in (EmployeeIterator, employee_generator):
        call_failed = False
        try:
            list(iter_obj(hacker))
        except IterationError as e:
            call_failed = True
            assert str(e) == _ITERATION_MESSAGE
        assert call_failed is True


if __name__ == "__main__":
    main()